In [1]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
%cd ..

synth_count = 2
from domblar.domblar import Domblar
d = Domblar(
    synth_count=synth_count,
    context='dexed'
)

/Users/gexahedron/devel/domblar


In [8]:
# Music in (various) Fifths

# - create phrases
# - make variable sus values? when we have breaks: https://www.youtube.com/watch?v=O6bMvj-yFFs
# - add timbre change
# - add dynamic tonality
# btw, music in similar motion uses fourths

# import time

# from domblar.edo import Scale, get_freq
# from domblar.music_math import freq_to_midi
from domblar.transformations import transpose
from domblar.edo import Scale

def set_synths():
    # for i in range(synth_count):
    d.set_synth(0, 'harpic3')
    d.set_synth(1, 'harpic3')
set_synths()

edo = 12
scale = list(range(edo))

m_scale = Scale([0, 2, 3, 5, 7, 8, 10], edo)  # F Dorian / C Aeolian
#                C  D  Eb F  G  Ab Bb

def gen_fifth(idx):
    return transpose([m_scale[idx], m_scale[idx + 4]], 0)

chords = []
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5]:
    chords.append(gen_fifth(idx))
# for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    # chords.append(gen_fifth(idx))

dur = 0.17
d.play(chords, scale, edo,
       synth_idx=[0, 1],
       dur=dur, sus=dur * 0.99)


In [9]:
# with added dynamic tuning, proof-of-concept
# say we calculate notes dynamically
# but we need a reference point
# what will it be?
# and also - scale would change from [0, 2, 3, 5, 7, 8, 10]
# to a list of vectors; octaves and fifths
# say we fix C (i guess it's better to fix F)
# wait, it's easier to fix Ab, because the scale is Ab Lydian
# [(-3, 4), (-4, 6), (-1, 1), (-2, 3), (-3, 5), (0, 0), (-1, 2)]

from domblar.transformations import transpose
from domblar.edo import Scale

def set_synths():
    d.set_synth(0, 'harpic3')
    d.set_synth(1, 'harpic3')
set_synths()

edo = 12
scale = list(range(edo))

class Scale_2d():
    def __init__(self, notes) -> None:
        self.notes = notes

    def __getitem__(self, note_idx):
        in_oct = self.notes[note_idx % len(self.notes)]
        oct_idx = note_idx // len(self.notes)
        return (in_oct[0] + oct_idx, in_oct[1])

m_scale = Scale_2d(
    [(-3, 4), (-4, 6), (-1, 1), (-2, 3), (-3, 5), (0, 0), (-1, 2)])  # F Dorian / C Aeolian

def gen_fifth(idx):
    return [(m_scale[idx][0] - 1, m_scale[idx][1]),
            (m_scale[idx + 4][0] - 1, m_scale[idx + 4][1])]

chords = []
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    chords.append(gen_fifth(idx))
for idx in [3, 4, 5, 6, 7, 6, 5, 4]:
    chords.append(gen_fifth(idx))

dur = 0.17

WESTERN_EDO = 12
A4_FREQ = 440.0
Ab4_FREQ = A4_FREQ * (2 ** (-1 / WESTERN_EDO))

iteration = 0
def get_freq_2d(note, scale, edo):
    global iteration
    freq = None
    if isinstance(note, tuple):
        OCTAVE = 2
        fifths = [
            # 3/2,
            # 2 ** (7/12),
            # 2 ** (9/16),
            2 ** (13/22),
            # 2 ** (11/19),
            # 2 ** (18/31),
            # 2 ** (31/53),
        ]
        fifths.sort()
        iteration += 1
        FIFTH = fifths[iteration % len(fifths)]
        freq = Ab4_FREQ * (OCTAVE ** note[0]) * (FIFTH ** note[1])
    return freq

def play_2d(chords, scale, edo, client,
         dur=0.25, sus=None, delay=None, synth_idx=[0], rep=1,
         muls=[], amps=[]):
    import time
    if chords and muls:
        assert len(chords) == len(muls)
    if chords and amps:
        assert len(chords) == len(amps)
    if type(rep) is list:
        assert len(synth_idx) == len(rep)
    else:
        rep = [rep] * len(synth_idx)
    last_reps = [0] * len(synth_idx)
    for chord_idx, chord in enumerate(chords):
        if type(chord) is tuple:
            chord = list(chord)
        elif type(chord) is not list:
            chord = [chord]
        for note_idx, note in enumerate(chord):
            freq = get_freq_2d(note, scale, edo)
            send_note_dur = dur
            if sus:
                send_note_dur = sus
            if muls:
                send_note_dur *= muls[chord_idx]
            amp = 1.0
            if amps:
                amp = amps[chord_idx]
            timetag = time.time()
            client.send_note(
                synth_idx[note_idx] + last_reps[note_idx],
                freq=freq, dur=send_note_dur, amp=amp,
                timetag=timetag, channel=0)
            last_reps[note_idx] = (last_reps[note_idx] + 1) % rep[note_idx]
            if delay:
                time.sleep(delay)
        sleep_dur = dur
        if muls:
            sleep_dur *= muls[chord_idx]
        time.sleep(sleep_dur)

play_2d(chords, scale, edo, d.client,
        synth_idx=[0, 1],
        dur=dur, sus=dur * 0.99)


In [96]:
finetuning = 0

try:
    finetuning
except Exception as e:
    d.open_editor(0)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(0)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(0)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            d.load_preset(i)
    else:
        d.print_params(0)

In [ ]:
# cleanup
d.stop_server()